# 1. Import Library

Thêm các thư viện cần thiết cho chương trình

In [1]:
import os
import pandas as pd
import difflib
from collections import defaultdict
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# 2. Load Processed Dataset

Định nghĩa đường dẫn dữ liệu đã được làm sạch trước đó

In [2]:
multivariate_data_path = r"Multivariate-data.csv"

Đọc dữ liệu

In [3]:
df = pd.read_csv(multivariate_data_path)

Xem thông tin tổng quát của dữ liệu

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Địa phương                                  378 non-null    object 
 1   Năm                                         378 non-null    int64  
 2   Vùng kinh tế                                378 non-null    object 
 3   Tiền đầu tư giáo dục                        295 non-null    float64
 4   Dân số trung bình                           315 non-null    float64
 5   Mật độ dân số                               315 non-null    float64
 6   Chỉ số Giáo dục                             189 non-null    float64
 7   Chỉ số phát triển con người                 189 non-null    float64
 8   Chỉ số sức khỏe                             189 non-null    float64
 9   Chỉ số thu nhập                             189 non-null    float64
 10  Điểm thi đại h

Kiểm tra số dữ liệu bị null

In [5]:
df.isnull().sum()


Địa phương                                      0
Năm                                             0
Vùng kinh tế                                    0
Tiền đầu tư giáo dục                           83
Dân số trung bình                              63
Mật độ dân số                                  63
Chỉ số Giáo dục                               189
Chỉ số phát triển con người                   189
Chỉ số sức khỏe                               189
Chỉ số thu nhập                               189
Điểm thi đại học trung bình của địa phương      0
dtype: int64

Xem mô tả dữ liệu

In [6]:
df.describe()

,Năm,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Chỉ số Giáo dục,Chỉ số phát triển con người,Chỉ số sức khỏe,Chỉ số thu nhập,Điểm thi đại học trung bình của địa phương
count,378.000000,295.000000,315.000000,315.000000,189.000000,189.000000,189.000000,189.000000,378.000000
mean,2020.500000,0.932986,7.135180,5.730927,0.603884,0.687116,0.818688,0.659238,6.026370
std,1.710089,0.859497,0.578922,0.993517,0.070226,0.049934,0.037324,0.065808,0.967508
min,2018.000000,0.000001,5.743003,3.912023,0.441000,0.565000,0.703000,0.532000,3.690686
25%,2019.000000,0.526814,6.774658,4.927254,0.546000,0.656000,0.807000,0.616000,5.351556
50%,2020.500000,0.733744,7.117287,5.605802,0.607000,0.683000,0.826000,0.647000,5.935970
75%,2022.000000,1.007667,7.430696,6.507278,0.652000,0.712000,0.842000,0.684000,6.716655
max,2023.000000,10.308810,9.147371,8.407602,0.790000,0.799000,0.872000,0.906000,8.283732


In [7]:
df.head()

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Chỉ số Giáo dục,Chỉ số phát triển con người,Chỉ số sức khỏe,Chỉ số thu nhập,Điểm thi đại học trung bình của địa phương
0,Hà Giang,2018,Vùng trung du và miền núi phía Bắc,0.434603,6.741110,4.670490,0.462,0.565,0.734,0.532,4.382694
1,Hà Giang,2019,Vùng trung du và miền núi phía Bắc,0.485911,6.754721,4.682131,0.482,0.579,0.737,0.545,4.215383
2,Hà Giang,2020,Vùng trung du và miền núi phía Bắc,0.422106,6.768769,4.700480,0.500,0.591,0.745,0.555,5.221484
3,Hà Giang,2021,Vùng trung du và miền núi phía Bắc,0.474379,6.787946,4.718499,NaN,NaN,NaN,NaN,5.680538
4,Hà Giang,2022,Vùng trung du và miền núi phía Bắc,0.554896,6.794273,4.727388,NaN,NaN,NaN,NaN,5.299501


# 3. Feature Engineering

## 3.1 Elimination of missing data for the year 2023 due to significant information gaps and split the year 2022 for test data set.


In [8]:
missing_data = pd.DataFrame()
years = [2018, 2019, 2020, 2021, 2022, 2023]
for year in years:
    missing_percentage = (df[df['Năm'] == year].isnull().sum() / len(df[df['Năm'] == year])) * 100
    missing_data[year] = missing_percentage


print(missing_data)

                                                2018      2019      2020  \
Địa phương                                  0.000000  0.000000  0.000000   
Năm                                         0.000000  0.000000  0.000000   
Vùng kinh tế                                0.000000  0.000000  0.000000   
Tiền đầu tư giáo dục                        6.349206  6.349206  6.349206   
Dân số trung bình                           0.000000  0.000000  0.000000   
Mật độ dân số                               0.000000  0.000000  0.000000   
Chỉ số Giáo dục                             0.000000  0.000000  0.000000   
Chỉ số phát triển con người                 0.000000  0.000000  0.000000   
Chỉ số sức khỏe                             0.000000  0.000000  0.000000   
Chỉ số thu nhập                             0.000000  0.000000  0.000000   
Điểm thi đại học trung bình của địa phương  0.000000  0.000000  0.000000   

                                                  2021        2022   2023  
Địa phương 

Excluded data for 2023 due to complete lack of data in the Education Investment, Average Population and Population Density columns

In [9]:
df = df[df['Năm'] != 2023]

df.head(10)

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Chỉ số Giáo dục,Chỉ số phát triển con người,Chỉ số sức khỏe,Chỉ số thu nhập,Điểm thi đại học trung bình của địa phương
0,Hà Giang,2018,Vùng trung du và miền núi phía Bắc,0.434603,6.741110,4.670490,0.462,0.565,0.734,0.532,4.382694
1,Hà Giang,2019,Vùng trung du và miền núi phía Bắc,0.485911,6.754721,4.682131,0.482,0.579,0.737,0.545,4.215383
2,Hà Giang,2020,Vùng trung du và miền núi phía Bắc,0.422106,6.768769,4.700480,0.500,0.591,0.745,0.555,5.221484
3,Hà Giang,2021,Vùng trung du và miền núi phía Bắc,0.474379,6.787946,4.718499,NaN,NaN,NaN,NaN,5.680538
4,Hà Giang,2022,Vùng trung du và miền núi phía Bắc,0.554896,6.794273,4.727388,NaN,NaN,NaN,NaN,5.299501
6,Cao Bằng,2018,Vùng trung du và miền núi phía Bắc,0.750797,6.270421,4.368308,0.558,0.624,0.777,0.560,4.488041
7,Cao Bằng,2019,Vùng trung du và miền núi phía Bắc,0.704093,6.274574,4.369448,0.571,0.633,0.778,0.570,4.575898
8,Cao Bằng,2020,Vùng trung du và miền núi phía Bắc,0.785310,6.278690,4.382027,0.581,0.641,0.778,0.584,5.246310
9,Cao Bằng,2021,Vùng trung du và miền núi phía Bắc,0.646162,6.295672,4.394449,NaN,NaN,NaN,NaN,5.801083
10,Cao Bằng,2022,Vùng trung du và miền núi phía Bắc,0.804120,6.297201,4.394449,NaN,NaN,NaN,NaN,5.306257


In [10]:
df['Năm'].unique()

array([2018, 2019, 2020, 2021, 2022], dtype=int64)

Split 2022 data for test set

In [11]:
df[df['Năm'] == 2022].to_csv('test_data.csv')
df = df[df['Năm'] != 2022]

df.head(10)

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Chỉ số Giáo dục,Chỉ số phát triển con người,Chỉ số sức khỏe,Chỉ số thu nhập,Điểm thi đại học trung bình của địa phương
0,Hà Giang,2018,Vùng trung du và miền núi phía Bắc,0.434603,6.741110,4.670490,0.462,0.565,0.734,0.532,4.382694
1,Hà Giang,2019,Vùng trung du và miền núi phía Bắc,0.485911,6.754721,4.682131,0.482,0.579,0.737,0.545,4.215383
2,Hà Giang,2020,Vùng trung du và miền núi phía Bắc,0.422106,6.768769,4.700480,0.500,0.591,0.745,0.555,5.221484
3,Hà Giang,2021,Vùng trung du và miền núi phía Bắc,0.474379,6.787946,4.718499,NaN,NaN,NaN,NaN,5.680538
6,Cao Bằng,2018,Vùng trung du và miền núi phía Bắc,0.750797,6.270421,4.368308,0.558,0.624,0.777,0.560,4.488041
7,Cao Bằng,2019,Vùng trung du và miền núi phía Bắc,0.704093,6.274574,4.369448,0.571,0.633,0.778,0.570,4.575898
8,Cao Bằng,2020,Vùng trung du và miền núi phía Bắc,0.785310,6.278690,4.382027,0.581,0.641,0.778,0.584,5.246310
9,Cao Bằng,2021,Vùng trung du và miền núi phía Bắc,0.646162,6.295672,4.394449,NaN,NaN,NaN,NaN,5.801083
12,Lạng Sơn,2018,Vùng trung du và miền núi phía Bắc,0.092488,6.654024,4.536570,0.626,0.667,0.803,0.590,5.488299
13,Lạng Sơn,2019,Vùng trung du và miền núi phía Bắc,0.209428,6.662877,4.543295,0.614,0.668,0.805,0.605,5.642575


In [12]:
df['Năm'].unique()

array([2018, 2019, 2020, 2021], dtype=int64)

## 3.2 Dimensionality Reduction

In ra phần trăm dữ liệu bị thiếu

In [13]:
missing_percentage = (df.isnull().sum() / len(df)) * 100

print("Phần trăm dữ liệu bị thiếu cho mỗi cột:")
missing_percentage

Phần trăm dữ liệu bị thiếu cho mỗi cột:


Địa phương                                     0.000000
Năm                                            0.000000
Vùng kinh tế                                   0.000000
Tiền đầu tư giáo dục                           6.349206
Dân số trung bình                              0.000000
Mật độ dân số                                  0.000000
Chỉ số Giáo dục                               25.000000
Chỉ số phát triển con người                   25.000000
Chỉ số sức khỏe                               25.000000
Chỉ số thu nhập                               25.000000
Điểm thi đại học trung bình của địa phương     0.000000
dtype: float64

Eliminate data columns with large percentages of data

In [14]:
df = df.drop(columns=['Chỉ số Giáo dục', 'Chỉ số phát triển con người', 'Chỉ số sức khỏe', 'Chỉ số thu nhập'])

df.head(10)

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
0,Hà Giang,2018,Vùng trung du và miền núi phía Bắc,0.434603,6.741110,4.670490,4.382694
1,Hà Giang,2019,Vùng trung du và miền núi phía Bắc,0.485911,6.754721,4.682131,4.215383
2,Hà Giang,2020,Vùng trung du và miền núi phía Bắc,0.422106,6.768769,4.700480,5.221484
3,Hà Giang,2021,Vùng trung du và miền núi phía Bắc,0.474379,6.787946,4.718499,5.680538
6,Cao Bằng,2018,Vùng trung du và miền núi phía Bắc,0.750797,6.270421,4.368308,4.488041
7,Cao Bằng,2019,Vùng trung du và miền núi phía Bắc,0.704093,6.274574,4.369448,4.575898
8,Cao Bằng,2020,Vùng trung du và miền núi phía Bắc,0.785310,6.278690,4.382027,5.246310
9,Cao Bằng,2021,Vùng trung du và miền núi phía Bắc,0.646162,6.295672,4.394449,5.801083
12,Lạng Sơn,2018,Vùng trung du và miền núi phía Bắc,0.092488,6.654024,4.536570,5.488299
13,Lạng Sơn,2019,Vùng trung du và miền núi phía Bắc,0.209428,6.662877,4.543295,5.642575


## 3.3 Handling Anomalous Data

In [15]:
df.describe()

,Năm,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
count,252.000000,236.000000,252.000000,252.000000,252.000000
mean,2019.500000,0.901783,7.130592,5.726445,5.838629
std,1.120259,0.679560,0.578784,0.993803,0.985430
min,2018.000000,0.000001,5.743003,3.912023,3.690686
25%,2018.750000,0.517495,6.771677,4.925435,5.050203
50%,2019.500000,0.723869,7.116840,5.605802,5.755935
75%,2020.250000,1.014320,7.428295,6.507278,6.459526
max,2021.000000,4.245046,9.129954,8.390268,8.283732


It can be seen that the data on investment in education has the lowest value of 0.000001, equivalent to 10 million VND.

==> Therefore, it is necessary to handle these unusual data

Consider the data with educational investment less than 100 million VND

In [16]:
df[df['Tiền đầu tư giáo dục'] < 1e-5]

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
14,Lạng Sơn,2020,Vùng trung du và miền núi phía Bắc,0.000001,6.670399,4.553877,6.424082


Maybe Lang Son's education investment data in 2020 is the only data with a value less than 100 million VND. Therefore it is necessary to process this data. Here I use the linear interpolation method to recalculate this unusual data point.

In [17]:
from scipy import interpolate

years = [2018, 2019, 2021]

# Lấy giá trị của các năm trước
values = [df.loc[(df["Địa phương"] == "Lạng Sơn") & (df["Năm"] == year), 'Tiền đầu tư giáo dục'].values[0] for year in years]

# Tạo hàm nội suy
interpolation_function = interpolate.interp1d(years, values, kind='linear', fill_value="extrapolate")

# Ước lượng giá trị cho năm 2022 bằng cách sử dụng hàm nội suy
estimated_value_2020 = interpolation_function(2020)

print(estimated_value_2020)

df.loc[(df["Địa phương"] == "Lạng Sơn") & (df["Năm"] == 2020), 'Tiền đầu tư giáo dục'] = estimated_value_2020

0.1387315


In [18]:
df[df['Tiền đầu tư giáo dục'] < 1e-5]

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương


## 3.4 Handling NaN Values


In [19]:
missing_percentage = (df.isnull().sum() / len(df)) * 100

print("Phần trăm dữ liệu bị thiếu cho mỗi cột:")
missing_percentage

Phần trăm dữ liệu bị thiếu cho mỗi cột:


Địa phương                                    0.000000
Năm                                           0.000000
Vùng kinh tế                                  0.000000
Tiền đầu tư giáo dục                          6.349206
Dân số trung bình                             0.000000
Mật độ dân số                                 0.000000
Điểm thi đại học trung bình của địa phương    0.000000
dtype: float64

In [20]:
df[df['Tiền đầu tư giáo dục'].isnull()]

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
24,Phú Thọ,2018,Vùng trung du và miền núi phía Bắc,NaN,7.279043,6.016450,6.057704
25,Phú Thọ,2019,Vùng trung du và miền núi phía Bắc,NaN,7.290566,6.028279,6.455052
26,Phú Thọ,2020,Vùng trung du và miền núi phía Bắc,NaN,7.301067,6.037871,7.265861
27,Phú Thọ,2021,Vùng trung du và miền núi phía Bắc,NaN,7.318234,6.056784,7.948110
180,Thừa Thiên - Huế,2018,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.025894,5.436164,4.335906
181,Thừa Thiên - Huế,2019,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.029530,5.438079,4.941810
182,Thừa Thiên - Huế,2020,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.033145,5.433722,5.852023
183,Thừa Thiên - Huế,2021,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.050816,5.451038,6.023467
228,Bình Thuận,2018,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.112572,5.040194,5.563746
229,Bình Thuận,2019,Vùng Bắc Trung Bộ và duyên hải miền Trung,NaN,7.116638,5.043425,5.898866


See the list of localities with null education investment data

In [21]:
df[df['Tiền đầu tư giáo dục'].isnull()]['Địa phương'].unique()

array(['Phú Thọ', 'Thừa Thiên - Huế', 'Bình Thuận', 'Bình Phước'],
      dtype=object)

Process educational investment data by assigning them data on the average educational investment amount for each year and economic region of that locality.

In [22]:
provinces = df[df['Tiền đầu tư giáo dục'].isnull()]['Địa phương'].unique()
years = df[df['Tiền đầu tư giáo dục'].isnull()]['Năm'].unique()

for province in provinces:
    for year in years:
        # Lấy tên vùng kinh tế của địa phương
        region = df[df['Địa phương'] == province]['Vùng kinh tế'].iloc[0]
            
        average_investment = df[(df["Vùng kinh tế"] == region) & (df["Năm"] == year)]["Tiền đầu tư giáo dục"].mean()

        # Điền giá trị trung bình vào các ô null trong cùng một nhóm
        df.loc[(df["Địa phương"] == province) & (df["Năm"] == year) & (df["Tiền đầu tư giáo dục"].isnull()), "Tiền đầu tư giáo dục"] = average_investment


In [23]:
df.isnull().sum()

Địa phương                                    0
Năm                                           0
Vùng kinh tế                                  0
Tiền đầu tư giáo dục                          0
Dân số trung bình                             0
Mật độ dân số                                 0
Điểm thi đại học trung bình của địa phương    0
dtype: int64

## 3.5 Categorical Feature Encoding

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 375
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Địa phương                                  252 non-null    object 
 1   Năm                                         252 non-null    int64  
 2   Vùng kinh tế                                252 non-null    object 
 3   Tiền đầu tư giáo dục                        252 non-null    float64
 4   Dân số trung bình                           252 non-null    float64
 5   Mật độ dân số                               252 non-null    float64
 6   Điểm thi đại học trung bình của địa phương  252 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 15.8+ KB


Get a list of object data to use label encoding

In [25]:
object_data = df.select_dtypes(include='object')

In [26]:
object_data.dtypes

Địa phương      object
Vùng kinh tế    object
dtype: object

Perform label encoding for each object data column

In [27]:
from sklearn.preprocessing import LabelEncoder

# Tạo một bản sao của DataFrame
label_encoded_df = df.copy()

label_models = {}

for col in object_data.columns:
    label_models[col] = LabelEncoder()

    # Áp dụng Label Encoding cho các biến object
    label_encoded_df[col] = label_models[col].fit_transform(label_encoded_df[col])


Export and save csv file id table of object data columns

In [28]:
for col, model in label_models.items():
    data_maping = []

    for id, province in enumerate(model.classes_):
        data_maping.append({'id': id, 'province': province})
    data_maping = pd.DataFrame(data_maping)
    print(data_maping)
    file_name = 'province_id_map.csv' if col == 'Địa phương' else 'region_id_map.csv'
    data_maping.to_csv(file_name, index=False)

    id           province
0    0           An Giang
1    1  Bà Rịa - Vũng Tàu
2    2         Bình Dương
3    3         Bình Phước
4    4         Bình Thuận
..  ..                ...
58  58            Đà Nẵng
59  59            Đắk Lắk
60  60           Đắk Nông
61  61           Đồng Nai
62  62          Đồng Tháp

[63 rows x 2 columns]
   id                                   province
0   0  Vùng Bắc Trung Bộ và duyên hải miền Trung
1   1                            Vùng Tây Nguyên
2   2         Vùng trung du và miền núi phía Bắc
3   3                           Vùng Đông Nam Bộ
4   4               Vùng đồng bằng sông Cửu Long
5   5                   Vùng đồng bằng sông Hồng


In [29]:
label_encoded_df.head(10)

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
0,14,2018,2,0.434603,6.741110,4.670490,4.382694
1,14,2019,2,0.485911,6.754721,4.682131,4.215383
2,14,2020,2,0.422106,6.768769,4.700480,5.221484
3,14,2021,2,0.474379,6.787946,4.718499,5.680538
6,11,2018,2,0.750797,6.270421,4.368308,4.488041
7,11,2019,2,0.704093,6.274574,4.369448,4.575898
8,11,2020,2,0.785310,6.278690,4.382027,5.246310
9,11,2021,2,0.646162,6.295672,4.394449,5.801083
12,30,2018,2,0.092488,6.654024,4.536570,5.488299
13,30,2019,2,0.209428,6.662877,4.543295,5.642575


# 4. Feature Selection

### 4.1 Evaluate feature importance using RandomForestRegressor

In [30]:
from sklearn.ensemble import RandomForestRegressor

# Tạo một bản sao của DataFrame
df_copy = label_encoded_df.copy()

# Chia dữ liệu thành biến độc lập (X) và biến phụ thuộc (y)
X = df_copy.drop(columns=['Điểm thi đại học trung bình của địa phương'])  # Thay 'Điểm thi đại học trung bình của địa phương' bằng tên cột bạn muốn dự đoán
y = df_copy['Điểm thi đại học trung bình của địa phương']

# Tạo mô hình Random Forest
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

# Huấn luyện mô hình trên dữ liệu
rf_model.fit(X, y)

# Lấy tầm quan trọng của các đặc trưng
feature_importances = rf_model.feature_importances_

# Tạo một DataFrame để hiển thị tầm quan trọng của các đặc trưng
importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sắp xếp DataFrame theo tầm quan trọng giảm dần
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# In ra tầm quan trọng của các đặc trưng
print(importance_df)


                Feature  Importance
1                   Năm    0.448468
5         Mật độ dân số    0.237046
4     Dân số trung bình    0.114634
3  Tiền đầu tư giáo dục    0.091387
0            Địa phương    0.080382
2          Vùng kinh tế    0.028082


# 5. Generate and Save LabelEncoding Data to CSV

In [31]:
label_encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 375
Data columns (total 7 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Địa phương                                  252 non-null    int32  
 1   Năm                                         252 non-null    int64  
 2   Vùng kinh tế                                252 non-null    int32  
 3   Tiền đầu tư giáo dục                        252 non-null    float64
 4   Dân số trung bình                           252 non-null    float64
 5   Mật độ dân số                               252 non-null    float64
 6   Điểm thi đại học trung bình của địa phương  252 non-null    float64
dtypes: float64(4), int32(2), int64(1)
memory usage: 13.8 KB


In [32]:
label_encoded_df.describe()

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
count,252.00000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,31.00000,2019.500000,2.507937,0.915172,7.130592,5.726445,5.838629
std,18.22043,1.120259,1.793063,0.670197,0.578784,0.993803,0.985430
min,0.00000,2018.000000,0.000000,0.027806,5.743003,3.912023,3.690686
25%,15.00000,2018.750000,1.000000,0.526879,6.771677,4.925435,5.050203
50%,31.00000,2019.500000,2.000000,0.738661,7.116840,5.605802,5.755935
75%,47.00000,2020.250000,4.000000,1.039475,7.428295,6.507278,6.459526
max,62.00000,2021.000000,5.000000,4.245046,9.129954,8.390268,8.283732


In [33]:
label_encoded_df.head(10)

,Địa phương,Năm,Vùng kinh tế,Tiền đầu tư giáo dục,Dân số trung bình,Mật độ dân số,Điểm thi đại học trung bình của địa phương
0,14,2018,2,0.434603,6.741110,4.670490,4.382694
1,14,2019,2,0.485911,6.754721,4.682131,4.215383
2,14,2020,2,0.422106,6.768769,4.700480,5.221484
3,14,2021,2,0.474379,6.787946,4.718499,5.680538
6,11,2018,2,0.750797,6.270421,4.368308,4.488041
7,11,2019,2,0.704093,6.274574,4.369448,4.575898
8,11,2020,2,0.785310,6.278690,4.382027,5.246310
9,11,2021,2,0.646162,6.295672,4.394449,5.801083
12,30,2018,2,0.092488,6.654024,4.536570,5.488299
13,30,2019,2,0.209428,6.662877,4.543295,5.642575


In [34]:
output_csv_file = "label_encoder_engineered_data_final.csv"

# Xuất DataFrame thành tệp CSV
label_encoded_df.to_csv(output_csv_file, index=False)

print("Exported processed data to CSV file:", output_csv_file)


Exported processed data to CSV file: label_encoder_engineered_data_final.csv


# 6. Generate and Save OneHotEncoding Data to CSV

Prepare data One hot encoding

In [35]:
onehotencoder_df = pd.get_dummies(label_encoded_df, columns=['Vùng kinh tế', 'Địa phương'], prefix=['Vùng kinh tế', 'Địa phương'])

In [36]:
onehot_output_csv_file = "onehot_engineered_data_final.csv"

# Xuất DataFrame thành tệp CSV
onehotencoder_df.to_csv(onehot_output_csv_file, index=False)

print("Exported processed data to CSV file:", onehot_output_csv_file)

Exported processed data to CSV file: onehot_engineered_data_final.csv
